The DBLEU Loss follow this paper : https://openreview.net/forum?id=S1x2aiRqFX 

#Import library

In [ ]:
!nvidia-smi

Thu May 19 14:24:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/CP-Eng-3rd-year/Pattern Recog/Project/00 Library/')
!pip install --upgrade pythainlp
!pip install torchinfo
!pip install torchviz

     |████████████████████████████████| 11.5 MB 4.1 MB/s 
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4150 sha256=a7b3ea4b407d985d021dd5e01a3982320bebbde7a630fe3bc787ad034d8b2d0e
  Stored in directory: /root/.cache/pip/wheels/04/38/f5/dc4f85c3909051823df49901e72015d2d750bd26b086480ec2
Successfully built torchviz


In [ ]:
from data_processing_non import dictionary
from model_dataloader import EncoderDecoder

ModuleNotFoundError: ignored

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm 
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random


import torch
from torch import nn 
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn.functional as F
from torchvision import transforms, models
from torch.nn.utils.rnn import pad_sequence
from torchinfo import summary

from sklearn.model_selection import train_test_split

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#Data set and Data loader


In [ ]:
def get_identical_matrix(vector):
  identical_matrix = torch.eye(len(vector))

  for i in range(len(vector)):
    for j in range(len(vector)):
      identical_matrix[i, j] = vector[i] == vector[j]

  return identical_matrix

def get_n_gram_hadamard_operation(matrix, n = 4):
  outputs = []
  matrix_n = matrix
  outputs.append(matrix)

  w = matrix.shape[0]
  h = matrix.shape[1]

  for i in range(1, n):
    if w-i <= 0 or h-i <= 0:
      break
    matrix_n = matrix[:-i, :-i] * matrix_n[1:, 1:]
    outputs.append(matrix_n)

  return outputs 

def get_similarity_matrix(pred, caption):
  T = len(caption)
  output = torch.zeros(T, T)
  for i in range(T):
    for j in range(T):
      output[i, j] = pred[i, caption[j]].contiguous()
  return output

In [ ]:
class FlickrDataset_Bleu_loss(Dataset):
    def __init__(self, 
                 captions_files, 
                 image_dir = "/content/drive/MyDrive/CP-Eng-3rd-year/Pattern Recog/Project/00 Data/archive/Images/", 
                 transform=None,
                 dictionary=dictionary,
                 test = False):
      
      self.image_dir = image_dir

      if type(captions_files) is str:
        self.dfs = pd.read_csv(captions_files)
        self.dfs = [self.df]
      elif type(captions_files) is list:
        for i in range(len(captions_files)):
          if  type(captions_files[i]) is str:
            captions_files[i] = pd.read_csv(captions_files[i])
        self.dfs = captions_files

      self.transform = transform
      self.test = test
      self.img_id = []
      self.img = {}
      self.caption = {}
      self.dictionary = dictionary

      self.caption_n_gram_count = {}

      unique_image = self.dfs[0]['image'].unique()

      for i in tqdm(range(len(unique_image))):
        img_id = unique_image[i]
        if img_id in self.caption:
          continue
        self.img_id.append(img_id)
        self.caption[img_id] = []
        self.caption_n_gram_count[img_id] = []
      
      for df in self.dfs:

        for i in tqdm(range(len(df))):
          img_id = df.iloc[i]['image']
          caption = df.iloc[i]['tokenized_caption_th']

          caption = caption[1:-1]
          caption = caption.split(",")
          caption = [word[1:].replace("'", "") for word in caption] 
          caption = self.dictionary.numericalize(caption)
          caption = torch.tensor(caption)
          
          identical_matrix = get_identical_matrix(caption[1:-1])
          n_gram_indentical_matrices = get_n_gram_hadamard_operation(identical_matrix)
          n_gram_count = [torch.sum(matrix, dim = 1) for matrix in n_gram_indentical_matrices]
          self.caption[img_id].append(caption)
          self.caption_n_gram_count[img_id].append(n_gram_count)
        
        
    def __len__(self):
      return len(self.img_id)
    
    def __getitem__(self, idx): 

      img_id = self.img_id[idx]
      if img_id not in self.img:
        img_path = os.path.join(self.image_dir, img_id)
        img = Image.open(img_path).convert("RGB")
        self.img[img_id] = img

      img = self.img[img_id]
      randomized_index = random.randint(0, len(self.caption[img_id])-1)
      caption = self.caption[img_id][randomized_index]
      caption_n_gram_count = self.caption_n_gram_count[img_id][randomized_index]

      if self.transform is not None:
        transformed_img = self.transform(img)

      if not self.test:
        return transformed_img, caption, caption_n_gram_count, img

      return transformed_img, caption, caption_n_gram_count, img, self.caption[img_id]
      

In [ ]:
# Collate_fn for calling to transform a batch before return. 
class Collate_fn:
  def __init__(self, pad_value, batch_first=False):
    self.pad_value = pad_value       
    self.batch_first = batch_first    
        
  def __call__(self, batch):
    transformed_imgs = [item[0].unsqueeze(0) for item in batch]
    transformed_imgs = torch.cat(transformed_imgs, dim=0)    
        
    captions = [item[1] for item in batch]   
    captions = pad_sequence(captions, batch_first = self.batch_first, padding_value = self.pad_value)
    
    if len(batch[0]) > 4:
      return transformed_imgs, captions, [item[2] for item in batch], [item[3] for item in batch], [item[4] for item in batch]

    return transformed_imgs, captions, [item[2] for item in batch], [item[3] for item in batch]

In [ ]:
dataframe_path = "/content/drive/MyDrive/CP-Eng-3rd-year/Pattern Recog/Project/00 Data/archive/"

In [ ]:
train_captions_files = [dataframe_path+"captions_train.csv"]
val_captions_files = [dataframe_path+"captions_val.csv"]
test_captions_files = [dataframe_path+"captions_test.csv"]

image_size = 299

train_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])
])

val_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])
])

In [ ]:
batch_size = 32
train_flickr_bleu_dataset = FlickrDataset_Bleu_loss(transform = train_transform, captions_files = train_captions_files)
train_flickr_bleu_dataloader = torch.utils.data.DataLoader( train_flickr_bleu_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=True, 
                                                pin_memory=True, 
                                                collate_fn = Collate_fn(pad_value=0, batch_first = True)) 
val_flickr_bleu_dataset = FlickrDataset_Bleu_loss(transform = val_transform, captions_files = val_captions_files)
val_flickr_bleu_dataloader = torch.utils.data.DataLoader( val_flickr_bleu_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=True, 
                                                pin_memory=True, 
                                                collate_fn = Collate_fn(pad_value=0, batch_first = True)) 
test_flickr_bleu_dataset = FlickrDataset_Bleu_loss(transform = val_transform, captions_files = test_captions_files, test = True)
test_flickr_bleu_dataloader = torch.utils.data.DataLoader( test_flickr_bleu_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=True, 
                                                pin_memory=True, 
                                                collate_fn = Collate_fn(pad_value=0, batch_first = True)) 

In [ ]:
len(next(iter(test_flickr_bleu_dataloader)))

#Model

In [ ]:
embed_size=300
vocab_size = len(dictionary.stoi)
attention_dim=256
encoder_dim=2048
decoder_dim=512
learning_rate = 3e-4

In [ ]:
transformed_img, captions, caption_n_gram_count, img = next(iter(train_flickr_bleu_dataloader))
transformed_img, captions = transformed_img.to(device), captions.to(device)

In [ ]:
model = EncoderDecoder(embed_size, vocab_size, attention_dim,encoder_dim,decoder_dim).to(device)

In [ ]:
#best_path = "/content/drive/MyDrive/CP-Eng-3rd-year/Pattern Recog/Project/01 Model/Anon/baseline_attention_cross_ling_model_state.pth"
#best = torch.load(best_path)
#model.load_state_dict(best['state_dict'])

#BLEU Loss Function

In [ ]:
from torch.autograd import Variable
eps = 1e-6
class BLEU_loss(nn.Module):
  def __init__(self, N_max = 4):
    super().__init__()
    self.N_max = N_max 
        
  def forward(self, preds, captions, Cs):
    prec = torch.zeros(len(preds))
    BP = torch.zeros(len(preds))

    for index in range(len(preds)):
      caption = captions[index]

      stop_word_index = (caption == 2).nonzero()[0]
      caption = caption[1:stop_word_index]
      T = len(caption)

      pred = preds[index]
      text_output = torch.argmax(pred, dim = 1)
      if 2 in text_output:
        stop_word_index_pred = (text_output == 2).nonzero()[0]
        text_output = text_output[:stop_word_index_pred]
      else :
        text_output = text_output
      pred = pred[:stop_word_index]
      pred = nn.functional.softmax(pred, dim=1)

      similarity_matrix = get_similarity_matrix(pred, caption)
      Ms = get_n_gram_hadamard_operation(similarity_matrix) 

      C = Cs[index]
      Os = torch.tensor([0.0 for _ in range(self.N_max)])

      for n in range(self.N_max):
        if n >= len(Ms):
          break
        M = Ms[n]
        diag = torch.diagonal(M, 0)
        temp = 1 + torch.sum(M, dim = 1) - diag 

        appearance_number = C[n]
        min_part = appearance_number/temp
        min_part = torch.clamp(min_part, max = 1)

        #print(M.shape, appearance_number.shape, min_part.shape)
        temp = (M.T/appearance_number*min_part).T
        temp = torch.sum(temp)
        Os[n] = torch.log(temp/(T-n))

      number_word_real = len(caption)
      number_word_pred = len(text_output)
      if number_word_real > number_word_pred:
        BP[index] = -torch.log(torch.exp(torch.tensor(1-number_word_real/(number_word_pred+eps))))
      prec[index] = -torch.sum(Os)/len(Ms)
      

    BP = torch.mean(BP) 
    prec = torch.mean(prec) 

    return BP+prec

#Training


In [ ]:
num_epoch = 25

In [ ]:
bleu_loss_fn = BLEU_loss()
criterion = nn.CrossEntropyLoss(ignore_index=dictionary.stoi["<PAD>"])
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, factor = 0.1, patience = 0, min_lr = 1e-7)

In [ ]:
best_path = "/content/drive/MyDrive/CP-Eng-3rd-year/Pattern Recog/Project/01 Model/Anon/bleu_loss_model_state.pth"

def save_model(model,num_iteration):
    model_state = {
        'num_epochs':num_iteration,
        'embed_size':embed_size,
        'vocab_size':vocab_size,
        'attention_dim':attention_dim,
        'encoder_dim':encoder_dim,
        'decoder_dim':decoder_dim,
        'state_dict':model.state_dict()
    }
    torch.save(model_state, best_path)
    print(best_path)

In [ ]:
import math

best_val = float('inf')
lrs = []
train_losses = []
val_losses = []



for i in tqdm(range(num_epoch)): 

  model.train()
  total_train_loss = 0 
  count = 0 
  weight = 0.35*math.exp(-i/10)
  print(0.65+weight, 0.35-weight)

  for transformed_img, captions, caption_n_gram_count, _ in tqdm(train_flickr_bleu_dataloader):
  
    optimizer.zero_grad()
    transformed_img, captions = transformed_img.to(device), captions.to(device)

    outputs, attentions = model(transformed_img, captions)
    #bleu_loss = bleu_loss_fn(outputs, captions, caption_n_gram_count)
    criterion_loss = criterion(outputs.reshape(-1, vocab_size), captions[:,1:].reshape(-1))

    train_loss = criterion_loss 
    train_loss.backward()

    total_train_loss += train_loss
    count+= 1
    optimizer.step()

  total_train_loss /= count
  scheduler.step(total_train_loss)
  lrs.append(optimizer.param_groups[0]['lr'])
  train_losses.append(train_loss.item())

  model.eval()
  total_val_loss = 0 
  count = 0 
  with torch.no_grad():
    for transformed_img, captions, caption_n_gram_count, _ in val_flickr_bleu_dataloader:
      transformed_img, captions = transformed_img.to(device), captions.to(device)
      outputs, attentions = model(transformed_img, captions)
      bleu_loss = bleu_loss_fn(outputs, captions, caption_n_gram_count)
      criterion_loss = criterion(outputs.reshape(-1, vocab_size), captions[:,1:].reshape(-1))

      val_loss = criterion_loss * (0.65+weight) + bleu_loss * (0.35-weight)

      total_val_loss += val_loss
      count += 1

    total_val_loss /= count
    val_losses.append(total_val_loss.item())
    if total_val_loss < best_val:
      best_val = total_val_loss
      save_model(model, i)

    transformed_img, captions, _, img = next(iter(val_flickr_bleu_dataloader)) 
    transformed_img, captions = transformed_img.to(device), captions.to(device)
    text_output, _ = model.generate_text(transformed_img)

    print(weight)
    print("LR : ", lrs[-1],"Train loss : ", total_train_loss, "Val loss : ",total_val_loss)
    for i in range(5):
      print("Real: ", "".join([ dictionary.itos[i] for i in captions[i].cpu().detach().numpy() ]))
      print("Prodicted: ", "".join(text_output[i]))
      plt.imshow(img[i])
      plt.show()
    print()

  

In [ ]:
#Way to optimize, consider the full matrix with <PAD> but assign the prob or something to be zero for ignoring
#Start and end might be considered

#BLEU eval

In [ ]:
def show_image_for_testing(all_captions, text_output, img):
  print()
  print("Predicted: ", "".join(text_output))
  for caption in all_captions:
    caption = [dictionary.itos[token.item()] for token in caption]
    reference.append(caption)
    print("Real: ", "".join(caption))
  print(sentence_bleu(reference, text_output))
  plt.imshow(img)
  plt.show()

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction, corpus_bleu
avg_bleu_score = 0
count = 0
max_count = 10
model.eval()

ref = list()
predict = list()


with torch.no_grad():
  for transformed_img, captions, caption_n_gram_count, img, all_captions in tqdm(test_flickr_bleu_dataloader):
    transformed_img, captions = transformed_img.to(device), captions.to(device)
    text_output, _ = model.generate_text(transformed_img)
    for i in range(len(all_captions)):
      reference = []
      for caption in all_captions[i]:
        reference.append([dictionary.itos[token.item()] for token in caption if token not in [0, 1, 2, 3]])
      candidate = [ token for token in text_output[i] if dictionary.stoi[token] not in [0, 1, 2, 3]]

      ref.append(reference)
      predict.append(candidate)

      bleu = sentence_bleu(reference, candidate)
      if bleu <0.2:
        show_image_for_testing(all_captions[i], text_output[i], img[i])


In [ ]:
print("Avg TEST BLEU Score:", corpus_bleu(ref, predict))